In [7]:
#Import library
from qiskit import*
from qiskit import Aer
from qiskit.quantum_info import partial_trace, Statevector
from qiskit.extensions.simulator import snapshot

import numpy as np
from numpy import square
from math import sqrt
import matplotlib.pyplot as plt
from random import randint

# identity function
def f0(qc,ip,op):
    qc.i(op)

# copy function
def f1(qc,ip,op):
    qc.cx(ip,op)

# copy and invert function
def f2(qc,ip,op):
    qc.cx(ip,op)
    qc.x(op)

# invert function
def f3(qc,ip,op):
    qc.x(op)

def oracle_1_ip(qc):
    t = randint(0,3)
    print("Secret function:",t)
    qc.barrier()
    if (t == 0):
        f0(qc,0,1)
    elif (t == 1):
        f1(qc,0,1)
    elif (t == 2):
        f2(qc,0,1)
    else:
        f3(qc,0,1)
    qc.barrier()

q_input = QuantumRegister(1, name = "a")
q_output = QuantumRegister(1, name = "c")
classical = ClassicalRegister(1, name = "classical")
qc = QuantumCircuit(q_input, q_output, classical)
qc.h(q_input)
print(qc.draw())

oracle_1_ip(qc)
qc.measure(q_output,classical)
print(qc.draw())

backend = Aer.get_backend('qasm_simulator')
sim = execute(qc, backend)
result = sim.result()
counts = result.get_counts()

if '0' in counts.keys():
    if counts['0'] == 1024:
        print("f0")
    else:
        print("f1 or f2")
else:
    print("f3")

             ┌───┐
        a_0: ┤ H ├
             └───┘
        c_0: ─────
                  
classical: 1/═════
                  
Secret function: 1
             ┌───┐ ░       ░    
        a_0: ┤ H ├─░───■───░────
             └───┘ ░ ┌─┴─┐ ░ ┌─┐
        c_0: ──────░─┤ X ├─░─┤M├
                   ░ └───┘ ░ └╥┘
classical: 1/═════════════════╩═
                              0 
f1 or f2


# 2 input 

In [8]:
# identity function
def f0(qc,ip_1,ip_2,op):
    qc.i(op)
    
# invert function
def f1(qc,ip_1,ip_2,op):
    qc.x(op)
    
# copy function(2)
def f2(qc,ip,op):
    qc.cx(ip,op)

# copy and invert function(2)
def f3(qc,ip,op):
    qc.cx(ip,op)
    qc.x(op)

# AND function
def f4(qc,ip_1,ip_2,op):
    qc.ccx(ip_1,ip_2,op)

# NAND function
def f5(qc,ip_1,ip_2,op):
    qc.ccx(ip_1,ip_2,op)
    qc.x(op)
    
# OR function
def f6(qc,ip_1,ip_2,op):
    qc.cx(ip_1,op)
    qc.cx(ip_2,op)
    qc.ccx(ip_1,ip_2,op)

# NOR function
def f7(qc,ip_1,ip_2,op):
    qc.cx(ip_1,op)
    qc.cx(ip_2,op)
    qc.ccx(ip_1,ip_2,op)
    qc.x(op)

# Implication_A function
def f8(qc,ip_1,ip_2,op):
    qc.x(ip_2)
    qc.ccx(ip_1,ip_2,op)

# Implication_B function
def f9(qc,ip_1,ip_2,op):
    qc.x(ip_1)
    qc.ccx(ip_1,ip_2,op)

# Inhibition_A function
def f10(qc,ip_1,ip_2,op):
    qc.x(ip_2)
    qc.cx(ip_1,op)
    qc.cx(ip_2,op)
    qc.ccx(ip_1,ip_2,op)
    
# Inhibition_B function
def f11(qc,ip_1,ip_2,op):
    qc.x(ip_1)
    qc.cx(ip_1,op)
    qc.cx(ip_2,op)
    qc.ccx(ip_1,ip_2,op)

# Ex-or
def f12(qc,ip_1,ip_2,op):
    qc.cx(ip_1,op)
    qc.cx(ip_2,op)
# Ex-Nor
def f13(qc,ip_1,ip_2,op):
    qc.cx(ip_1,op)
    qc.cx(ip_2,op)
    qc.x(op)

In [9]:
def oracle_2_ip(qc,t):
    #t = randint(0,3)
    print("Secret function:",t)
    qc.barrier()
    if (t == 0):
        f0(qc,0,1,2)
    elif (t == 1):
        f1(qc,0,1,2)
    elif (t == 2):
        f2(qc,0,2)
    elif (t == 3):
        f2(qc,1,2)
    elif (t == 4):
        f3(qc,0,2)
    elif (t == 5):
        f3(qc,1,2)
    elif (t == 6):
        f4(qc,0,1,2)
    elif (t == 7):
        f5(qc,0,1,2)
    elif (t == 8):
        f6(qc,0,1,2)
    elif (t == 9):
        f7(qc,0,1,2)
    elif (t == 10):
        f8(qc,0,1,2)
    elif (t == 11):
        f9(qc,0,1,2)
    elif (t == 12):
        f10(qc,0,1,2)
    elif (t == 13): 
        f11(qc,0,1,2)
    elif (t == 14):
        f12(qc,0,1,2)
    else:
        f13(qc,0,1,2) 
    qc.barrier()

In [10]:
def output_qubit_state(circuit):
    """Get the statevector for the first qubit, discarding the rest."""
    # get the full statevector of all qubits
    full_statevector = Statevector(circuit)

    # get the density matrix for the output qubit by taking the partial trace
    partial_density_matrix = partial_trace(full_statevector, [0,1])

    # extract the statevector out of the density matrix
    partial_statevector = np.diagonal(partial_density_matrix)
    return abs(partial_statevector)

In [11]:
initial_state_0 = [sqrt(3/4),sqrt(1/4)]
initial_state_1 = [sqrt(4/5), sqrt(1/5)]
for i in range(16):
    statevector1_arr = np.empty([8]).astype(complex)
    state =[]
    probs_0 = 0;
    probs_1 = 0;
    q_input = QuantumRegister(2, name = "a")
    q_output = QuantumRegister(1, name = "c")
    #classical = ClassicalRegister(1, name = "classical")
    qc = QuantumCircuit(q_input, q_output)
    qc.initialize(initial_state_0, 0) # Apply initialisation operation to the 0th qubit
    qc.initialize(initial_state_1, 1) # Apply initialisation operation to the 0th qubit
    qc.h(q_input)
    oracle_2_ip(qc,i)
    
    qc.snapshot("qbsnap", qubits=[2])
    backend = Aer.get_backend('statevector_simulator')
    snapshot1 = execute(qc, backend).result().data()['snapshots']['statevector']['qbsnap']
    
    for i in range(8):
        statevector1_arr[i] = (snapshot1[0][i])
    statevector = abs(statevector1_arr)**2
    for i in range(4):
        probs_0 += statevector[i];
    for i in range(4,8):
        probs_1 += statevector[i];
    state.append(probs_0)
    state.append(probs_1)
    
    print(state)

Secret function: 0
[1.0000000000000004, 0.0]
Secret function: 1
[0.0, 1.0000000000000004]
Secret function: 2
[0.9330127018922197, 0.06698729810778072]
Secret function: 3
[0.9000000000000005, 0.10000000000000003]
Secret function: 4
[0.06698729810778072, 0.9330127018922197]
Secret function: 5
[0.10000000000000003, 0.9000000000000005]
Secret function: 6
[0.9933012701892224, 0.006698729810778072]
Secret function: 7
[0.006698729810778072, 0.9933012701892224]
Secret function: 8
[0.8397114317029978, 0.16028856829700266]
Secret function: 9
[0.16028856829700266, 0.8397114317029978]
Secret function: 10
[0.9397114317029978, 0.060288568297002645]
Secret function: 11
[0.9066987298107786, 0.09330127018922196]
Secret function: 12
[0.09330127018922196, 0.9066987298107786]
Secret function: 13
[0.060288568297002645, 0.9397114317029979]
Secret function: 14
[0.8464101615137759, 0.1535898384862246]
Secret function: 15
[0.1535898384862246, 0.8464101615137759]


In [12]:
initial_state_0 = [sqrt(3/4),sqrt(1/4)]
initial_state_1 = [sqrt(4/5), sqrt(1/5)]
for i in range(16):
    statevector1_arr = np.empty([8]).astype(complex)
    state =[]
    probs_0 = 0;
    probs_1 = 0;
    q_input = QuantumRegister(2, name = "a")
    q_output = QuantumRegister(1, name = "c")
    #classical = ClassicalRegister(1, name = "classical")
    qc = QuantumCircuit(q_input, q_output)
    qc.initialize(initial_state_0, 0) # Apply initialisation operation to the 0th qubit
    qc.initialize(initial_state_1, 1) # Apply initialisation operation to the 0th qubit
    qc.h(q_input)
    oracle_2_ip(qc,i)
    print(output_qubit_state(qc))

Secret function: 0
[1. 0.]
Secret function: 1
[0. 1.]
Secret function: 2
[0.9330127 0.0669873]
Secret function: 3
[0.9 0.1]
Secret function: 4
[0.0669873 0.9330127]
Secret function: 5
[0.1 0.9]
Secret function: 6
[0.99330127 0.00669873]
Secret function: 7
[0.00669873 0.99330127]
Secret function: 8
[0.83971143 0.16028857]
Secret function: 9
[0.16028857 0.83971143]
Secret function: 10
[0.93971143 0.06028857]
Secret function: 11
[0.90669873 0.09330127]
Secret function: 12
[0.09330127 0.90669873]
Secret function: 13
[0.06028857 0.93971143]
Secret function: 14
[0.84641016 0.15358984]
Secret function: 15
[0.15358984 0.84641016]


# General initial input

In [29]:
# Create a sum free set which is power of 2 sequences
def sum_free_sequences(n):
    arr = []
    for i in range (1,n+1):
        arr.append(i**2)
    return arr;

In [30]:
sum_free_sequences(4)

[1, 4, 9, 16]

In [31]:
#Create initial input using sum free sequences
def initial_state(n):
    arr = []
    sumfree_arr = sum_free_sequences(n*2)
    for i in range (n):
        state = [sumfree_arr[i*2]/sqrt(sumfree_arr[i*2]**2 + sumfree_arr[i*2+1]**2) , sumfree_arr[i*2+1]/sqrt(sumfree_arr[i*2]**2 + sumfree_arr[i*2+1]**2)]
        arr.append(state)
    return arr

In [32]:
initial_state = initial_state(2)
initial_state

[[0.24253562503633297, 0.9701425001453319],
 [0.49026123963255896, 0.8715755371245493]]

In [33]:
initial_state_0 = initial_state[0]
initial_state_1 = initial_state[1]
for i in range(16):
    q_input = QuantumRegister(2, name = "a")
    q_output = QuantumRegister(1, name = "c")
    classical = ClassicalRegister(1, name = "classical")
    qc = QuantumCircuit(q_input, q_output, classical)
    qc.initialize(initial_state_0, 0) # Apply initialisation operation to the 0th qubit
    qc.initialize(initial_state_1, 1) # Apply initialisation operation to the 0th qubit
    qc.h(q_input)
    oracle_2_ip(qc,i)
    print(output_qubit_state(qc))

Secret function: 0
[1. 0.]
Secret function: 1
[0. 1.]
Secret function: 2
[0.73529412 0.26470588]
Secret function: 3
[0.9272997 0.0727003]
Secret function: 4
[0.26470588 0.73529412]
Secret function: 5
[0.0727003 0.9272997]
Secret function: 6
[0.9807558 0.0192442]
Secret function: 7
[0.0192442 0.9807558]
Secret function: 8
[0.68183802 0.31816198]
Secret function: 9
[0.31816198 0.68183802]
Secret function: 10
[0.75453831 0.24546169]
Secret function: 11
[0.9465439 0.0534561]
Secret function: 12
[0.0534561 0.9465439]
Secret function: 13
[0.24546169 0.75453831]
Secret function: 14
[0.70108221 0.29891779]
Secret function: 15
[0.29891779 0.70108221]


# Modified function 

In [34]:
def oracle_2_ip_modified(qc,t):
    arr = [];
    #t = randint(0,3)
    print("Secret function:",t)
    qc.barrier()
    if (t == 0):
        f0(qc,0,1,2)
        arr = [0, 0, 0]
    elif (t == 1):
        f1(qc,0,1,2)
        arr = [1, 0, 0]
    elif (t == 2):
        f2(qc,0,2)
        arr = [0, 1, 0]
    elif (t == 3):
        f2(qc,1,2)
        arr = [0, 1, 0]
    elif (t == 4):
        f3(qc,0,2)
        arr = [1, 1, 0]
    elif (t == 5):
        f3(qc,1,2)
        arr = [1, 1, 0]
    elif (t == 6):
        f4(qc,0,1,2)
        arr = [0, 0, 1]
    elif (t == 7):
        f5(qc,0,1,2)
        arr = [1, 0, 1]
    elif (t == 8):
        f6(qc,0,1,2)
        arr = [0, 2, 1]
    elif (t == 9):
        f7(qc,0,1,2)
        arr = [1, 2, 1]
    elif (t == 10):
        f8(qc,0,1,2)
        arr = [1, 0, 1]
    elif (t == 11):
        f9(qc,0,1,2)
        arr = [1, 0, 1]
    elif (t == 12):
        f10(qc,0,1,2)
        arr = [1, 2, 1]
    elif (t == 13): 
        f11(qc,0,1,2)
        arr = [1, 2, 1]
    elif (t == 14):
        f12(qc,0,1,2)
        arr = [0, 2, 0]
    else:
        f13(qc,0,1,2)
        arr = [1, 2, 0]
    qc.barrier()
    return arr

In [36]:
initial_state_0 = initial_state[0]
initial_state_1 = initial_state[1]
gate_sequences =[] #[X, CX, CCX]
for i in range(16):
    q_input = QuantumRegister(2, name = "a")
    q_output = QuantumRegister(1, name = "c")
    classical = ClassicalRegister(1, name = "classical")
    qc = QuantumCircuit(q_input, q_output, classical)
    qc.initialize(initial_state_0, 0) # Apply initialisation operation to the 0th qubit
    qc.initialize(initial_state_1, 1) # Apply initialisation operation to the 0th qubit
    qc.h(q_input)
    gate_sequences = oracle_2_ip_modified(qc,i)
    print(output_qubit_state(qc))
    print("Gate sequences")
    print(gate_sequences)

Secret function: 0
[1. 0.]
Gate sequences
[0, 0, 0]
Secret function: 1
[0. 1.]
Gate sequences
[1, 0, 0]
Secret function: 2
[0.73529412 0.26470588]
Gate sequences
[0, 1, 0]
Secret function: 3
[0.9272997 0.0727003]
Gate sequences
[0, 1, 0]
Secret function: 4
[0.26470588 0.73529412]
Gate sequences
[1, 1, 0]
Secret function: 5
[0.0727003 0.9272997]
Gate sequences
[1, 1, 0]
Secret function: 6
[0.9807558 0.0192442]
Gate sequences
[0, 0, 1]
Secret function: 7
[0.0192442 0.9807558]
Gate sequences
[1, 0, 1]
Secret function: 8
[0.68183802 0.31816198]
Gate sequences
[0, 2, 1]
Secret function: 9
[0.31816198 0.68183802]
Gate sequences
[1, 2, 1]
Secret function: 10
[0.75453831 0.24546169]
Gate sequences
[1, 0, 1]
Secret function: 11
[0.9465439 0.0534561]
Gate sequences
[1, 0, 1]
Secret function: 12
[0.0534561 0.9465439]
Gate sequences
[1, 2, 1]
Secret function: 13
[0.24546169 0.75453831]
Gate sequences
[1, 2, 1]
Secret function: 14
[0.70108221 0.29891779]
Gate sequences
[0, 2, 0]
Secret function: 15